In [1]:
import simpy
import random
import statistics

Each staff member can work as an independent server and handle each customer from the 1 beginning to the end.

In [2]:
wait_times = []

In [50]:
#set up the environmet
class Sharetea(object):
    def __init__(self, env, drink_type, num_cashiers, num_servers, num_ushers):
        self.env = env
        self.drinktype = simpy.Resource(env, drink_type)
        self.cashier = simpy.Resource(env, num_cashiers)
        self.server = simpy.Resource(env, num_servers)
        self.usher = simpy.Resource(env, num_ushers)
        
    #the time of ordering a drink 
    def order_drink(self,customer):
        yield self.env.timeout(random.randint(1,3))
    
    def sell_food(self, customer):
        yield self.env.timeout(random.randint(1,5))
    
    def check_ticket(self, customer):
        yield self.env.timeout(3/60)

In [44]:
def go_to_movies(env, customer, theater):
    # customer arrives at the theater
    arrival_time = env.now
    
    #waiting behavior
    #a cashier can only help one customer at a time
    with theater.cashier.request() as request:
        yield request
        yield env.process(theater.order_drink(customer))
        
    with theater.usher.request() as request:
        yield request
        yield env.process(theater.order_drink(customer))
        
    if random.choice([True, False]):
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(customer))
            
    #customer heads into the theater
    wait_times.append(env.now - arrival_time)

define a function to run the simulation

In [52]:
def run_theater(env, num_cashiers, num_servers, num_ushers):
    theater = Sharetea(env, num_cashiers, num_servers, num_ushers)
    
    #expect around 3 customer in line ready to buy tickets as soon as the box office opens
    for customer in range(3):
        env.process(go_to_movies(env, customer, theater))
        
    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new person
        
        customer += 1
        env.process(go_to_movies(env, customer, theater))
    

In [46]:
def get_user_input():
    num_cashiers = input("Input # of cashiers working: ")
    num_servers = input("Input # of servers working: ")
    num_ushers = input("Input # of ushers working: ")
    params = [num_cashiers, num_servers, num_ushers]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. The simulation will use default values:",
            "\n1 cashier, 1 server, 1 usher.",
        )
        params = [1, 1, 1]
    return params

In [47]:
def main():
  # Setup
  random.seed(42)
  num_cashiers, num_servers, num_ushers = get_user_input()

  # Run the simulation
  env = simpy.Environment()
  env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
  env.run(until=90)

  # View the results
  mins, secs = get_average_wait_time(wait_times)
  print(
      "Running simulation...",
      f"\nThe average wait time is {mins} minutes and {secs} seconds.",
  )

In [48]:
def get_average_wait_time(arrival_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

In [53]:
if __name__ == '__main__':
    main()

Input # of cashiers working: 1
Input # of servers working: 1
Input # of ushers working: 1
Running simulation... 
The average wait time is 44 minutes and 21 seconds.
